In [ ]:
# import numpy as np 
import pandas as pd  
import string
import itertools
from itertools import combinations
import math

#Prepare Data
Data = pd.read_excel("/TURF ANALYSIS/Data/Data.xlsx")
Data['M3_ITB'] = Data.M3_ITB.apply(lambda x: x[0: ].split(','))
res = Data.set_index(['Respondent_ID'])['M3_ITB'].apply(pd.Series).stack()
res = res.reset_index()
res.columns = ['Respondent_ID','level_1','Flavor']
Data_Bi = res.pivot_table(index ='Respondent_ID', columns ='Flavor' , aggfunc ="count" )
Data_Bi.columns = Data_Bi.columns.droplevel()  #Delete Multiindex
#Data ready to run TURF Analysis
Data_Bi = Data_Bi.replace(np.nan, 0)
#Get List of Flavors
Flavor_List = Data_Bi.columns.values.tolist()

def Response(Sample_Size):
    Cases = Data_Bi.sum()
    Cases = pd.DataFrame(Cases)
    Cases.reset_index(inplace = True)
    Cases.columns = ['Flavor','Total_Cases']
    Cases['Perc_Total_Cases'] = Cases['Total_Cases']/Sample_Size
    Cases.sort_values(['Total_Cases'], ascending = False, inplace= True)
    return Cases

Response(209)  


def result(combination, Sample_Size, top):
    comb =  list(combinations(Flavor_List, combination))    
    for items in comb:
        Data_Bi['_'.join(items)] = Data_Bi.loc[:, items].sum(1)

    Data_Bi_ = Data_Bi.replace(0, np.nan)
    Result = Data_Bi_.count()
    Result_Reach = pd.DataFrame(Result)    
    Result_Reach.reset_index(inplace = True)
    Result_Reach.columns = ['Flavor_Combinations','Reach']
    Result_Reach.sort_values(['Reach'], ascending = False, inplace = True)
    Result_Reach['Perc_Reach'] = (Result_Reach['Reach']/Sample_Size)*100
    Result_Reach['Combination_no'] = combination
    Result_Reach_top = Result_Reach.nlargest(top,['Reach'])
    return Result_Reach_top 

Final_result = pd.concat([result(combination=4,Sample_Size=209,top=5),result(combination=5,Sample_Size=209,top=5),result(combination=6,Sample_Size=209,top=5)])
    
Final_result.to_excel("/Final_result.xlsx")
